Para el EDA, solo trabajaré con el df de train, analice la cantidad de valores nulos y cantidad de datos, el test lo usaré en el apartado de los modelos, por lo que ya no estará en el EDA, para que este más limpio el código.

In [1]:
# Importar librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Confiugarción de gráficos 
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (10,5)

# Ignorar warnings
import warnings
warnings.filterwarnings('ignore')

print('Librerías importadas correctamente')

Librerías importadas correctamente


In [24]:
# Cargar datos
data_path = r'C:\Users\candy\Downloads\arkon_ds\data\train_set.csv'
df = pd.read_csv(data_path)






- **trip_id**: identificador único para el viaje
- **duration**: duración del viaje en minutos
- **start_time**: dia/hora donde en viaje inicia en formato ISO 8601 tiempo local
- **end_time**: dia/hora donde el viaje termina en formato ISO 8601 tiempo local
- **start_station**: la estación donde el viaje inició
- **start_lat**: la latitud de la estación donde el viaje se originó
- **start_lon**: la longitud de la estación donde el viaje se originó
- **end_station**: la estación donde el viaje terminó
- **end_lat**: la latitud de la estación donde terminó el viaje
- **end_lon**: la longitud de la estación donde terminó el viaje
- **bike_id**: un entero único que identifica la bicicleta
- **plan_duration**: número de días que el usuario tendrá el paso. 0 significa un viaje único
(Walk-up plan)
- **trip_route_category**: “Round trip” son viajes que empiezan y terminan en la misma
estación
- **passholder_type**: El nombre del plan de passholde

In [25]:
def resumen_dataframe(df):
    """
    Muestra información general del dataset, valores nulos y estadística descriptiva.
    """
    print("Información General:")
    display(df.info())  # Estructura del DataFrame
    
    print("\n Primeras Filas:")
    display(df.head())  # Primeras filas del dataset
    
    print("\n Valores Nulos:")
    display(df.isnull().sum()[df.isnull().sum() > 0])  # Muestra solo columnas con valores nulos
    
    print("\n Estadísticas Descriptivas:")
    display(df.describe())  # Estadísticas generales del dataset
    
    print("\n Distribución de Clases:")
    if "passholder_type" in df.columns:
        display(df["passholder_type"].value_counts())  # Verifica la distribución de la variable objetivo


resumen_dataframe(df)

Información General:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700000 entries, 0 to 699999
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   trip_id              700000 non-null  int64  
 1   duration             700000 non-null  int64  
 2   start_time           700000 non-null  object 
 3   end_time             700000 non-null  object 
 4   start_lat            694437 non-null  float64
 5   start_lon            694437 non-null  float64
 6   end_lat              681426 non-null  float64
 7   end_lon              681426 non-null  float64
 8   bike_id              700000 non-null  object 
 9   plan_duration        699792 non-null  float64
 10  trip_route_category  700000 non-null  object 
 11  passholder_type      697424 non-null  object 
 12  start_station        700000 non-null  int64  
 13  end_station          700000 non-null  int64  
dtypes: float64(5), int64(4), object(5)
memory usage

None


 Primeras Filas:


,trip_id,duration,start_time,end_time,start_lat,start_lon,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,start_station,end_station
0,101750280,35,2018-08-07 11:20:00,2018-08-07 11:55:00,33.748920,-118.275192,33.748920,-118.275192,06530,1.0,Round Trip,Walk-up,4127,4127
1,46560345,32,9/17/2017 17:51,9/17/2017 18:23,34.035679,-118.270813,34.047749,-118.243172,6683,0.0,One Way,Walk-up,3057,3062
2,120016336,6,2019-04-22 09:22:00,2019-04-22 09:28:00,34.046070,-118.233093,34.047749,-118.243172,06710,30.0,One Way,Monthly Pass,3022,3062
3,129547190,138,9/22/2019 11:27,9/22/2019 13:45,34.062580,-118.290092,34.059689,-118.294662,17068,1.0,One Way,One Day Pass,4304,4311
4,136619463,14,1/31/2020 17:11,1/31/2020 17:25,34.026291,-118.277687,34.021660,-118.278687,18841,30.0,One Way,Monthly Pass,4266,4443



 Valores Nulos:


start_lat           5563
start_lon           5563
end_lat            18574
end_lon            18574
plan_duration        208
passholder_type     2576
dtype: int64


 Estadísticas Descriptivas:


,trip_id,duration,start_lat,start_lon,end_lat,end_lon,plan_duration,start_station,end_station
count,7.000000e+05,700000.000000,694437.000000,694437.000000,681426.000000,681426.000000,699792.000000,700000.000000,700000.000000
mean,1.069468e+08,37.084979,34.044952,-118.253849,34.044175,-118.259183,44.928697,3499.720464,3489.726771
std,4.497342e+07,125.302510,0.325255,2.332640,0.299721,2.129781,92.816296,615.918795,613.040769
min,8.369648e+06,1.000000,33.710979,-118.495422,33.710979,-118.495422,0.000000,3000.000000,3000.000000
25%,7.538051e+07,7.000000,34.037460,-118.280952,34.037048,-118.280952,1.000000,3031.000000,3030.000000
50%,1.179410e+08,13.000000,34.046612,-118.256980,34.046520,-118.256981,30.000000,3064.000000,3064.000000
75%,1.404178e+08,26.000000,34.051941,-118.247162,34.050911,-118.246422,30.000000,4214.000000,4214.000000
max,1.794831e+08,1440.000000,55.705528,118.238258,55.705528,37.606541,999.000000,4594.000000,4594.000000



 Distribución de Clases:


passholder_type
Monthly Pass    394769
Walk-up         212426
One Day Pass     44455
Annual Pass      34092
Flex Pass        11604
Testing             78
Name: count, dtype: int64

In [ ]:
df.fillna(df.median(), inplace=True)  # Rellenar con la mediana
df.dropna(inplace=True)  # Eliminar filas con valores nulos


In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x="passholder_type", order=df["passholder_type"].value_counts().index, palette="coolwarm")
plt.xticks(rotation=45)
plt.title("Distribución de Tipos de Pase")
plt.show()

In [ ]:
df["start_time"] = pd.to_datetime(df["start_time"])
df["hour"] = df["start_time"].dt.hour

sns.histplot(df["hour"], bins=24, kde=True)
plt.title("Distribución de Viajes por Hora del Día")
plt.xlabel("Hora del Día")
plt.ylabel("Número de Viajes")
plt.show()

In [ ]:
top_stations = df["start_station"].value_counts().head(10)

plt.figure(figsize=(10, 5))
sns.barplot(x=top_stations.values, y=top_stations.index, palette="Blues_r")
plt.xlabel("Número de Viajes")
plt.ylabel("Estación")
plt.title("Top 10 Estaciones con Más Viajes")
plt.show()

In [ ]:
git add notebooks/EDA.ipynb
git commit -m "Análisis Exploratorio de Datos (EDA) completado"
git push origin main